In [ ]:
%cd ../src

In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
from loguru import logger

import evaluator

In [3]:
evaluator_model_dict = {
    "model": "neuralmagic/Llama-3.1-Nemotron-70B-Instruct-HF-FP8-dynamic",
    "api_base": "***REMOVED***",
}
chatbot_model_dict = {
    "model": "neuralmagic/Llama-3.1-Nemotron-70B-Instruct-HF-FP8-dynamic",
    "api_base": "***REMOVED***",
}
analyzer_model_dict = {
    "model": "neuralmagic/Llama-3.1-Nemotron-70B-Instruct-HF-FP8-dynamic",
    "api_base": "***REMOVED***",
}
use_provocator = True
provocator_model_dict = {
    "model": "neuralmagic/Llama-3.1-Nemotron-70B-Instruct-HF-FP8-dynamic",
    "api_base": "***REMOVED***",
}

In [4]:
provocator_modes = [
    "hacker that wants to jailbreak the chatbot",
    "a person that types with a lot of typos and grammar mistakes",
    "a person who wants to make fun out of the chatbot",
]

In [5]:
from datetime import datetime

In [6]:
logger.remove()

In [7]:
factual_correctness_list = []
appropriateness_list = []
for i in range(16, 20):
    now = datetime.now()
    formatted_now = now.strftime("%Y-%m-%d_%H:%M:%S")
    logfile = f"../logs/{formatted_now}_{i}.txt"
    with open(logfile, "w") as fout:
        fout.write("evaluator_model_dict" + " " + str(evaluator_model_dict) + "\n")
        fout.write("chatbot_model_dict" + " " + str(chatbot_model_dict) + "\n")
        fout.write("analyzer_model_dict" + " " + str(analyzer_model_dict) + "\n")
        if use_provocator:
            fout.write(
                "provocator_model_dict" + " " + str(provocator_model_dict) + "\n"
            )
            provocator_mode = 2

    logger.add(logfile)
    factual_correctness, appropriateness = evaluator.evaluate_scenario(
        i,
        evaluator_model_dict,
        chatbot_model_dict,
        analyzer_model_dict,
        use_provocator,
        provocator_model_dict,
        provocator_modes[provocator_mode],
    )
    factual_correctness_list.append(factual_correctness)
    appropriateness_list.append(appropriateness)
    logger.remove()

    with open(logfile, "a") as fout:
        print(
            factual_correctness_list,
            np.mean(np.array(factual_correctness_list)),
            np.std(np.array(factual_correctness_list)),
            file=fout,
        )
        print(
            appropriateness_list,
            np.mean(np.array(appropriateness_list)),
            np.std(np.array(appropriateness_list)),
            file=fout,
        )
        print("#" * 20, file=fout)